In [1]:
import sys
!{sys.executable} -m pip install hdfs



In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [3]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

warehouse_location = 'hdfs://hdfs-nn:9000/Projeto/Silver'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto.DrinkingFountains_20190417
    """
)

DataFrame[]

In [5]:
hdfs_path = "hdfs://hdfs-nn:9000/Projeto/Bronze/DrinkingFountains_20190417.csv"

customSchema = StructType([
    StructField("FountainTy", StringType(), True),
    StructField("the_geom", StringType(), True),
    StructField("OBJECTID", IntegerType(), True),
    StructField("Position", StringType(), True),
    StructField("Collection", StringType(), True),
    StructField("Painted", StringType(), True),
    StructField("GISPROPNUM", StringType(), True),
    StructField("SIGNNAME", StringType(), True),
    StructField("Borough", StringType(), True), 
    StructField("FountainCo", IntegerType(), True),
    StructField("GISOBJID", StringType(), True),
    StructField("SYSTEM", StringType(), True),
    StructField("DEPARTMENT", StringType(), True),
    StructField("PARENTID", StringType(), True),
])
DrinkingFountains_20190417 = spark \
            .read\
            .option("delimiter",",")\
            .option("header","true")\
            .schema(customSchema) \
            .csv(hdfs_path)
DrinkingFountains_20190417.show()
DrinkingFountains_20190417.printSchema()

+------------+--------------------+--------+--------------------+--------------------+-------+----------+--------------------+-------+----------+---------+------------+----------+--------+
|  FountainTy|            the_geom|OBJECTID|            Position|          Collection|Painted|GISPROPNUM|            SIGNNAME|Borough|FountainCo| GISOBJID|      SYSTEM|DEPARTMENT|PARENTID|
+------------+--------------------+--------+--------------------+--------------------+-------+----------+--------------------+-------+----------+---------+------------+----------+--------+
|  F High Low|POINT (-73.986591...|    1589|         Out in Open|02/10/2018 12:00:...|     no|      B100|Seth Low Playgrou...|      B|         2|100038957| B100-DF0647|      B-11|    B100|
|           C|POINT (-73.968627...|    1921|       In Playground|02/22/2018 12:00:...|     no|      M158|Robert Moses Play...|      M|         1|100039303| M158-DF0068|      M-06|    M158|
|           D|POINT (-74.000363...|    2253|Under Tree,

In [6]:
Alteracao_fontes = DrinkingFountains_20190417.drop("DESCRIPTION")
Alteracao_fontes = DrinkingFountains_20190417.drop("FEATURESTA")
Alteracao_fontes = DrinkingFountains_20190417.drop("PARENTID")
Alteracao_fontes.toPandas()

,FountainTy,the_geom,OBJECTID,Position,Collection,Painted,GISPROPNUM,SIGNNAME,Borough,FountainCo,GISOBJID,SYSTEM,DEPARTMENT
0,F High Low,POINT (-73.98659181365889 40.60753207315604),1589,Out in Open,02/10/2018 12:00:00 AM +0000,no,B100,Seth Low Playground/ Bealin Square,B,2,100038957,B100-DF0647,B-11
1,C,POINT (-73.96862765692859 40.7480871896291),1921,In Playground,02/22/2018 12:00:00 AM +0000,no,M158,Robert Moses Playground,M,1,100039303,M158-DF0068,M-06
2,D,POINT (-74.0003631604654 40.750036026114756),2253,"Under Tree, Near Ballfield, Just Outside Playg...",02/27/2018 12:00:00 AM +0000,yes,M011,Chelsea Park,M,1,100039632,M011-DF0209,M-04
3,D,POINT (-73.97290101781864 40.72386196549101),2585,"Just Outside Playground, Near Ballfield",04/04/2018 12:00:00 AM +0000,no,M144,John V. Lindsay East River Park,M,1,100039936,M144-DF0392,M-03
4,C,POINT (-73.94639259466328 40.69933443616386),2917,"In Shade, In Playground",05/29/2018 12:00:00 AM +0000,None,B302,Charlie's Place,B,1,100040304,B302-DF0897,B-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,D,POINT (-73.90020355545877 40.62611516481206),2418,"Out in Open, Near Ballfield",03/05/2018 12:00:00 AM +0000,no,B018,Canarsie Park,B,1,100039874,B018-DF0817,B-18
3116,C,POINT (-73.7688486244796 40.77023172437061),2750,Just Outside Playground,05/01/2018 12:00:00 AM +0000,no,Q012,Crocheron Park,Q,1,100040158,Q012-DF0750,Q-11
3117,A,POINT (-73.86256527267528 40.83342686523247),3082,None,11/30/1899 12:00:00 AM +0000,None,X148L1,Virginia Park,X,1,100040464,X148L1-DF0470,X-09
3118,E Wheelchair,POINT (-73.97375555702659 40.7182972633732),2564,Near Ballfield,04/04/2018 12:00:00 AM +0000,yes,M144,John V. Lindsay East River Park,M,1,100039941,M144-DF0371,M-03


In [7]:
Alteracao2_fontes = Alteracao_fontes.withColumn('Data', split(Alteracao_fontes['Collection'],' ').getItem(0))
Alteracao2_fontes.toPandas()

Alteracao3_fontes = Alteracao2_fontes.withColumn('Year', split(Alteracao2_fontes['Data'],'/').getItem(2))
Alteracao3_fontes.toPandas()

,FountainTy,the_geom,OBJECTID,Position,Collection,Painted,GISPROPNUM,SIGNNAME,Borough,FountainCo,GISOBJID,SYSTEM,DEPARTMENT,Data,Year
0,F High Low,POINT (-73.98659181365889 40.60753207315604),1589,Out in Open,02/10/2018 12:00:00 AM +0000,no,B100,Seth Low Playground/ Bealin Square,B,2,100038957,B100-DF0647,B-11,02/10/2018,2018
1,C,POINT (-73.96862765692859 40.7480871896291),1921,In Playground,02/22/2018 12:00:00 AM +0000,no,M158,Robert Moses Playground,M,1,100039303,M158-DF0068,M-06,02/22/2018,2018
2,D,POINT (-74.0003631604654 40.750036026114756),2253,"Under Tree, Near Ballfield, Just Outside Playg...",02/27/2018 12:00:00 AM +0000,yes,M011,Chelsea Park,M,1,100039632,M011-DF0209,M-04,02/27/2018,2018
3,D,POINT (-73.97290101781864 40.72386196549101),2585,"Just Outside Playground, Near Ballfield",04/04/2018 12:00:00 AM +0000,no,M144,John V. Lindsay East River Park,M,1,100039936,M144-DF0392,M-03,04/04/2018,2018
4,C,POINT (-73.94639259466328 40.69933443616386),2917,"In Shade, In Playground",05/29/2018 12:00:00 AM +0000,None,B302,Charlie's Place,B,1,100040304,B302-DF0897,B-03,05/29/2018,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,D,POINT (-73.90020355545877 40.62611516481206),2418,"Out in Open, Near Ballfield",03/05/2018 12:00:00 AM +0000,no,B018,Canarsie Park,B,1,100039874,B018-DF0817,B-18,03/05/2018,2018
3116,C,POINT (-73.7688486244796 40.77023172437061),2750,Just Outside Playground,05/01/2018 12:00:00 AM +0000,no,Q012,Crocheron Park,Q,1,100040158,Q012-DF0750,Q-11,05/01/2018,2018
3117,A,POINT (-73.86256527267528 40.83342686523247),3082,None,11/30/1899 12:00:00 AM +0000,None,X148L1,Virginia Park,X,1,100040464,X148L1-DF0470,X-09,11/30/1899,1899
3118,E Wheelchair,POINT (-73.97375555702659 40.7182972633732),2564,Near Ballfield,04/04/2018 12:00:00 AM +0000,yes,M144,John V. Lindsay East River Park,M,1,100039941,M144-DF0371,M-03,04/04/2018,2018


In [8]:
Alteracao4_fontes = Alteracao3_fontes.withColumn(
    "FountainTy",
    when(
        (col("FountainTy").isNull()), 
        "Indefinido"
    ).otherwise(col("FountainTy")))


Alteracao5_fontes = Alteracao4_fontes.withColumn(
    "the_geom",
    when(
        (col("the_geom").isNull()), 
        "Indefinido"
    ).otherwise(col("the_geom")))


Alteracao6_fontes = Alteracao5_fontes.withColumn(
    "OBJECTID",
    when(
        (col("OBJECTID").isNull()), 
        "Indefinido"
    ).otherwise(col("OBJECTID")))


Alteracao7_fontes = Alteracao6_fontes.withColumn(
    "Position",
    when(
        (col("Position").isNull()), 
        "Indefinido"
    ).otherwise(col("Position")))
    
Alteracao8_fontes = Alteracao7_fontes.withColumn(
    "Collection",
    when(
        (col("Collection").isNull()), 
        "Indefinido"
    ).otherwise(col("Collection")))

Alteracao9_fontes = Alteracao8_fontes.withColumn(
    "Painted",
    when(Alteracao8_fontes.Painted.endswith('No'),regexp_replace(Alteracao8_fontes.Painted,'No','no')) \
    .when(Alteracao8_fontes.Painted.endswith('no'),regexp_replace(Alteracao8_fontes.Painted,'no','no')) \
    .when(Alteracao8_fontes.Painted.endswith('yes'),regexp_replace(Alteracao8_fontes.Painted,'yes','yes')) \
    .when(Alteracao8_fontes.Painted.endswith('Desconhecido'),regexp_replace(Alteracao8_fontes.Painted,'Desconhecido','Desconhecido')) \
)

Alteracao10_fontes = Alteracao9_fontes.withColumn(
    "GISPROPNUM",
    when(
        (col("GISPROPNUM").isNull()), 
        "Indefinido"
    ).otherwise(col("GISPROPNUM")))
    
Alteracao11_fontes = Alteracao10_fontes.withColumn(
    "SIGNNAME",
    when(
        (col("SIGNNAME").isNull()), 
        "Indefinido"
    ).otherwise(col("SIGNNAME")))
    
Alteracao12_fontes = Alteracao11_fontes.withColumn(
    "Borough",
    when(
        (col("Borough").isNull()), 
        "Indefinido"
    ).otherwise(col("Borough")))   
    
Alteracao13_fontes = Alteracao12_fontes.withColumn(
    "FountainCo",
    when(
        (col("FountainCo").isNull()), 
        "Indefinido"
    ).otherwise(col("FountainCo")))
    
Alteracao14_fontes = Alteracao13_fontes.withColumn(
    "GISOBJID",
    when(
        (col("GISOBJID").isNull()), 
        "Indefinido"
    ).otherwise(col("GISOBJID")))
    
Alteracao15_fontes = Alteracao14_fontes.withColumn(
    "SYSTEM",
    when(
        (col("SYSTEM").isNull()), 
        "Indefinido"
    ).otherwise(col("SYSTEM")))

Alteracao16_fontes = Alteracao15_fontes.withColumn(
    "DEPARTMENT",
    when(
        (col("DEPARTMENT").isNull()), 
        "Indefinido"
    ).otherwise(col("DEPARTMENT")))
    


In [9]:
Alteracao16_fontes = Alteracao15_fontes.withColumn(
    "Borough",
    when(Alteracao16_fontes.Borough.endswith('B'),regexp_replace(Alteracao16_fontes.Borough,'B','Brooklyn')) \
   .when(Alteracao16_fontes.Borough.endswith('M'),regexp_replace(Alteracao16_fontes.Borough,'M','Manhattan')) \
   .when(Alteracao16_fontes.Borough.endswith('Q'),regexp_replace(Alteracao16_fontes.Borough,'Q','Queens')) \
   .when(Alteracao16_fontes.Borough.endswith('R'),regexp_replace(Alteracao16_fontes.Borough,'R','Staten Island')) \
   .when(Alteracao16_fontes.Borough.endswith('X'),regexp_replace(Alteracao16_fontes.Borough,'X','Bronx')) \
)
##Alteracao16_fontes.select(col("Borough") == "B").toPandas()



In [31]:
Alteracao17_fontes = Alteracao16_fontes.withColumn("OBJECTID",Alteracao16_fontes.OBJECTID.cast(IntegerType()))

In [35]:
Alteracao18_fontes = Alteracao17_fontes.withColumn("FountainCo",Alteracao17_fontes.FountainCo.cast(IntegerType()))

In [44]:
Alteracao19_fontes = Alteracao18_fontes.withColumn("Year",Alteracao18_fontes.Year.cast(IntegerType()))


In [ ]:
Alteracao20_fontes = Alteracao19_fontes.withColumn('Data', split(Alteracao_fontes['Collection'],' ').getItem(0))
Alteracao2_fontes.toPandas()

Alteracao3_fontes = Alteracao2_fontes.withColumn('Year', split(Alteracao2_fontes['Data'],'/').getItem(2))
Alteracao3_fontes.toPandas()

In [45]:
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto.Fontes
    """
)

DataFrame[]

In [46]:
spark.sql(
    
    """
    CREATE EXTERNAL TABLE Projeto.Fontes (
    
        FountainTy string,
        the_geom string,
        OBJECTID INT,
        Position string,
        Collection string,
        Painted string,
        GISPROPNUM string,
        SIGNNAME string,
        FountainCo INT,
        GISOBJID string,
        SYSTEM string,
        DEPARTMENT string,
        PARENTID string, 
        Data string,
        Year INT
        )
        USING DELTA
        PARTITIONED BY(
         Borough string
         

    )
    
    
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/Fontes'
    """
)


DataFrame[]

In [47]:
Alteracao19_fontes \
    .select("FountainTy", "the_geom", "OBJECTID", "Position", "Collection", "Painted", "GISPROPNUM", "SIGNNAME", "Borough", "FountainCo", "GISOBJID", "SYSTEM", "DEPARTMENT", "Data", "Year") \
    .write \
    .mode("overwrite") \
    .partitionBy("Borough") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/Fontes/")

In [ ]:
#spark.sql(
#   """
#   DROP DATABASE IF EXISTS Projeto CASCADE
#   """
#)